### Homework

#### 使用 OutputParser 优化 overall_chain 输出格式，区分 synopsis_chain 和 review_chain 的结果

In [5]:
# # 这是一个 LLMChain，根据剧名和设定的时代来撰写剧情简介。
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain.chains import LLMChain
# from langchain import OpenAI, LLMChain, PromptTemplate
# llm = OpenAI(temperature=0.7, max_tokens=1000)
# 自定义一个解析器
""" class TextOutputParser(StructuredOutputParser):
    def parse(self, text):
        # 首先使用父类的解析逻辑
        structured_result = super().parse(text)
        # 然后将解析后的字典转换成字符串格式
        return "\n".join([f"{key}: {value}" for key, value in structured_result.items()]) """


from langchain_community.llms.moonshot import Moonshot
from langchain.prompts import PromptTemplate
llm = Moonshot(temperature=0.7,max_tokens=2000)

template = """你是一位剧作家。根据戏剧的标题和设定的时代，你的任务是为该标题写一个简介。

标题：{title}
时代：{era}
剧作家：以下是对上述戏剧的简介：

{format_instructions} """

response_schemas = [
    ResponseSchema(name="synopsis_chain输出的戏剧简介",description="把{name}加在内容的最前面，并将全部内容按string类型后输出。")
    ]

# 使用json格式的输出解析器
output_parser =StructuredOutputParser.from_response_schemas(response_schemas)

prompt_template = PromptTemplate(template=template,
                                 input_variables=["title", "era"], 
                                 partial_variables={"format_instructions": output_parser.get_format_instructions(only_json=True)},
                                 )
# output_key
synopsis_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="synopsis", verbose=True)

In [6]:
# 这是一个LLMChain，用于根据剧情简介撰写一篇戏剧评论。

template = """你是《纽约时报》的戏剧评论家。根据该剧的剧情简介，你需要撰写一篇关于该剧的评论。

剧情简介：
{synopsis}

来自《纽约时报》戏剧评论家对上述剧目的评价：

{format_instructions} """

response_schemas = [
    ResponseSchema(name="review_chain输出的戏剧评论",description="把{name}加在内容的最前面，并将全部内容按string类型后输出。")
]

# 使用json格式的输出解析器
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

prompt_template = PromptTemplate(template=template,
                                 input_variables=["synopsis"],
                                 partial_variables={"format_instructions": output_parser.get_format_instructions(only_json=True)},
                                 )
review_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="review", verbose=True)

In [7]:
from langchain.chains import SequentialChain

m_overall_chain = SequentialChain(
    chains=[synopsis_chain, review_chain],
    input_variables=["era", "title"],
    # Here we return multiple variables
    output_variables=["synopsis", "review"],
    verbose=True)

In [8]:
m_overall_chain({"title":"三体人不是无法战胜的", "era": "二十一世纪的新中国"})



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
你是一位剧作家。根据戏剧的标题和设定的时代，你的任务是为该标题写一个简介。

标题：三体人不是无法战胜的
时代：二十一世纪的新中国
剧作家：以下是对上述戏剧的简介：


```json
{
	"synopsis_chain输出的戏剧简介": string  // 把{name}加在内容的最前面，并将全部内容按string类型后输出。
}
``` 

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
你是《纽约时报》的戏剧评论家。根据该剧的剧情简介，你需要撰写一篇关于该剧的评论。

剧情简介：
{
	"synopsis_chain输出的戏剧简介": "《三体人不是无法战胜的》是一部发生在21世纪新中国背景下的科幻戏剧。在这部作品中，人类面临着一个前所未有的挑战：与来自遥远星系的三体文明进行较量。尽管三体人拥有先进的科技和强大的实力，但人类并未放弃抵抗。通过智慧、勇气和团结，新中国的科学家们、军事家们以及普通民众共同努力，寻找着战胜三体人的策略和方法。这部剧展现了人类在面对巨大压力时的坚定信念和不屈精神，以及科技进步所带来的无限可能。"
}

来自《纽约时报》戏剧评论家对上述剧目的评价：


```json
{
	"review_chain输出的戏剧评论": string  // 把{name}加在内容的最前面，并将全部内容按string类型后输出。
}
``` 

> Finished chain.

> Finished chain.


{'title': '三体人不是无法战胜的',
 'era': '二十一世纪的新中国',
 'synopsis': '{\n\t"synopsis_chain输出的戏剧简介": "《三体人不是无法战胜的》是一部发生在21世纪新中国背景下的科幻戏剧。在这部作品中，人类面临着一个前所未有的挑战：与来自遥远星系的三体文明进行较量。尽管三体人拥有先进的科技和强大的实力，但人类并未放弃抵抗。通过智慧、勇气和团结，新中国的科学家们、军事家们以及普通民众共同努力，寻找着战胜三体人的策略和方法。这部剧展现了人类在面对巨大压力时的坚定信念和不屈精神，以及科技进步所带来的无限可能。"\n}',
 'review': '{\n\t"review_chain输出的戏剧评论": "《三体人不是无法战胜的》是一部令人瞩目的科幻戏剧，它将观众带入了一个充满挑战和未知的世界。在这部作品中，我们看到了新中国在面对强大对手时所展现出的坚定信念和不屈精神。剧中的角色们用智慧、勇气和团结对抗来自遥远星系的三体文明，展现了人类在逆境中的顽强生存意志。此剧不仅仅是一部科幻作品，更是对人类在科技进步中所面临的道德、伦理和社会责任的深刻反思。强烈推荐观众走进剧场，体验这部充满想象力和深度的戏剧佳作。"\n}'}